Numa etapa posterior da análise, decidi investigar se seria viável identificar o sexo do molusco apenas a partir de suas medidas físicas. Minha intuição inicial (influenciada por exemplos de outras espécies, como a humana) era a de que machos e fêmeas apresentariam diferenças mensuráveis em tamanho ou massa, o que permitiria um processo de classificação razoável.

Para testar essa hipótese, utilizei todas as variáveis do conjunto de dados como preditoras, exceto o próprio atributo de sexo. Transformei esse atributo por meio de *one-hot encoding*, obtendo três categorias de resposta: “F”, “M” e “I”. Vale destacar que a categoria “I” (*infant*) não corresponde a um sexo biológico, mas sim a indivíduos jovens demais para que seja possível determinar seu sexo, uma condição que influencia fortemente o processo de classificação e justifica sua presença como rótulo separado.

Como o problema envolve apenas três classes, considerei a regressão logística uma escolha natural para iniciar a modelagem. Além disso, implementei também um classificador de floresta aleatória, por ser um método robusto e capaz de capturar relações mais complexas entre os atributos, servindo como uma boa base de comparação para o desempenho dos modelos lineares.


# Configuração do Ambiente

In [61]:
!pip install ucimlrepo


In [89]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib  import pyplot as plt
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier


# Preparação dos Dados

In [63]:
abalone = fetch_ucirepo(id=1)

abalone_df = pd.concat([abalone.data.features, abalone.data.targets], axis=1)
y = abalone_df["Sex"]
y= pd.get_dummies(y)
X = pd.concat([abalone.data.features, abalone.data.targets], axis=1)
X.drop(columns = ['Sex'],inplace = True)


In [64]:
y.head(10)

,F,I,M
0,False,False,True
1,False,False,True
2,True,False,False
3,False,False,True
4,False,True,False
5,False,True,False
6,True,False,False
7,True,False,False
8,False,False,True
9,True,False,False


In [65]:
X.head(10)

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
5,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8
6,0.530,0.415,0.150,0.7775,0.2370,0.1415,0.330,20
7,0.545,0.425,0.125,0.7680,0.2940,0.1495,0.260,16
8,0.475,0.370,0.125,0.5095,0.2165,0.1125,0.165,9
9,0.550,0.440,0.150,0.8945,0.3145,0.1510,0.320,19


# Classificação Macho, Femea e Infants

## Preparo dos Dados

Antes de prosseguir com a etapa de modelagem, precisei verificar o formato do vetor de respostas y, que representa as classes que desejo prever. Para isso, comecei imprimindo sua dimensão.

Esse comando me permite confirmar se y está armazenado como um vetor unidimensional — formato esperado pelos classificadores — ou se ainda mantém a estrutura resultante do one-hot encoding, que gera uma matriz com uma coluna para cada classe.

Em seguida, incluí uma verificação para corrigir automaticamente esse problema caso ele ocorra

In [66]:
print("y shape:", y.shape)


if len(y.shape) > 1:
    y = np.argmax(y, axis=1)
print("y shape:", y.shape)

y shape: (4177, 3)
y shape: (4177,)


Para x, utilizei o StandardScaler, que transforma cada atributo para ter média zero e desvio-padrão igual a um:

In [67]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

## Regressão Logística

O procedimento implementa um pipeline completo para treinamento e avaliação de um modelo de Regressão Logística aplicado à classificação multiclasse. Inicialmente, as variáveis preditoras são padronizadas por meio de `StandardScaler`, assegurando que todas apresentem média zero e variância unitária, o que favorece a estabilidade numérica e a convergência do algoritmo de otimização.

Em seguida, os dados são particionados em conjuntos de treino (80%) e teste (20%), utilizando estratificação para preservar a distribuição das classes. O modelo base de Regressão Logística é então submetido a um processo de validação cruzada estratificada (StratifiedKFold) combinado com Grid Search, no qual diferentes valores do hiperparâmetro de regularização `C` e distintos solvers são avaliados. A seleção do modelo ótimo é realizada com base na métrica de acurácia.

O melhor estimador resultante é ajustado ao conjunto de treino completo e posteriormente avaliado no conjunto de teste. São calculados a acurácia global e a matriz de confusão, permitindo derivar a acurácia por classe a partir da razão entre verdadeiros positivos e o total de amostras de cada categoria. Essa abordagem possibilita avaliar tanto o desempenho geral quanto a capacidade discriminativa do modelo em cada classe individual.


In [68]:
def treinar_logistic_regression(X, y):
    # Padronização
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)

    # Divisão treino-teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # Modelo base
    log_reg = LogisticRegression(max_iter=1000, random_state=42)

    # Hiperparâmetros para busca
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    }

    # Validação cruzada estratificada
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Grid Search
    grid_search = GridSearchCV(
        estimator=log_reg,
        param_grid=param_grid,
        scoring='accuracy',
        cv=cv,
        verbose=1,
        n_jobs=-1
    )

    # Treinamento
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Predição no conjunto de teste
    y_pred = best_model.predict(X_test)

    # Resultados principais
    print("Melhores hiperparâmetros:", grid_search.best_params_)
    print("Acurácia no conjunto de teste:", accuracy_score(y_test, y_pred))

    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])

    # Acurácia por classe
    category_accuracies = cm.diagonal() / cm.sum(axis=1)
    categories = ["F", "I", "M"]

    for category, accuracy in zip(categories, category_accuracies):
        print(f"Accuracy for {category}: {accuracy:.2f}")

    # Retornar modelo e métricas se quiser usar depois
    return {
        "best_model": best_model,
        "best_params": grid_search.best_params_,
        "accuracy": accuracy_score(y_test, y_pred),
        "confusion_matrix": cm,
        "class_accuracy": dict(zip(categories, category_accuracies)),
    }
resultado = treinar_logistic_regression(X, y)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Melhores hiperparâmetros: {'C': 1, 'solver': 'lbfgs'}
Acurácia no conjunto de teste: 0.5538277511961722
Accuracy for F: 0.33
Accuracy for I: 0.82
Accuracy for M: 0.52


Com base nos resultados obtidos, observa-se que a Regressão Logística apresenta desempenho moderado na tarefa de classificação do sexo dos abalones, alcançando cerca de 55% de acurácia global. A análise por classe revela um comportamento assimétrico: enquanto a categoria “Infant” é identificada com boa precisão (82%), as classes “Female” e “Male” apresentam desempenhos substancialmente inferiores (33% e 52%, respectivamente).

Esse padrão sugere que os atributos físicos disponíveis são insuficientes para discriminar de forma consistente entre machos e fêmeas adultos, possivelmente devido à grande sobreposição entre suas características morfométricas. Por outro lado, indivíduos classificados como infant possuem dimensões mais distintivas, o que facilita a identificação pelo modelo. Esses resultados indicam que métodos mais complexos ou a inclusão de variáveis adicionais seriam necessários para melhorar a separação entre os sexos adultos.


## Random Forest

A função implementa o treinamento e avaliação de um modelo Random Forest para a tarefa de classificação do sexo do molusco a partir de atributos físicos.

Inicialmente, define-se uma grade de hiperparâmetros para os componentes principais da floresta (número de árvores, profundidade máxima e critério de divisão) os quais são otimizados por meio de Grid Search combinado com validação cruzada estratificada em cinco dobras.

Após o ajuste, seleciona-se o modelo que obteve maior acurácia média na validação. Esse modelo é então aplicado ao conjunto de teste para estimar o desempenho fora da amostra.

Por fim, calcula-se a acurácia global e a acurácia por classe com base na matriz de confusão, permitindo avaliar a capacidade discriminativa do modelo para cada categoria de sexo.


In [77]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

def treinar_random_forest(X_train, y_train, X_test, y_test):

    # Modelo base
    rf = RandomForestClassifier(random_state=42)

    # Grade de hiperparâmetros
    param_grid = {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10],
    }

    # Validação cruzada estratificada
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Grid Search
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        scoring="accuracy",
        cv=cv,
        verbose=1,
        n_jobs=-1
    )

    # Ajuste do modelo
    grid_search.fit(X_train, y_train)

    # Melhor modelo encontrado
    best_model = grid_search.best_estimator_

    # Predição no conjunto de teste
    y_pred = best_model.predict(X_test)

    # Resultados
    print("Melhores hiperparâmetros:", grid_search.best_params_)
    print("Acurácia no conjunto de teste:", accuracy_score(y_test, y_pred))

    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])

    # Acurácia por classe
    class_accuracies = cm.diagonal() / cm.sum(axis=1)
    classes = ["F", "I", "M"]

    for cls, acc in zip(classes, class_accuracies):
        print(f"Acurácia para {cls}: {acc:.2f}")

    return best_model, y_pred, cm
resultado = treinar_random_forest(X_train, y_train, X_test, y_test)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Melhores hiperparâmetros: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Acurácia no conjunto de teste: 0.5418660287081339
Acurácia para F: 0.38
Acurácia para I: 0.77
Acurácia para M: 0.48



Com base nos resultados obtidos, observa-se que o desempenho do Random Forest reforça a interpretação já indicada pela análise exploratória e pela regressão logística. A acurácia global do modelo permanece moderada (≈0.54), e, novamente, as classes F e M apresentam desempenho insatisfatório (0.38 e 0.48, respectivamente). Em contraste, a classe I mantém desempenho significativamente superior (0.77), indicando que o modelo consegue distinguir com muito mais facilidade os indivíduos jovens (*infants*) do que diferenciar machos e fêmeas adultos.




## Conclusão

Essa assimetria na performance dos modelos confirma que os atributos físicos disponíveis capturam bem as diferenças estruturais entre animais jovens e adultos, mas não capturam diferenças relevantes entre os sexos. Assim, tanto a análise estatística quanto os experimentos de classificação convergem para a mesma conclusão: abalones adultos não apresentam distinções físicas suficientemente marcantes entre machos e fêmeas, inviabilizando uma classificação eficiente baseada apenas em medidas morfométricas.

Portanto, os resultados indicam que a tarefa de predição do sexo não é adequada com as variáveis disponíveis, enquanto a distinção entre indivíduos jovens e adultos é claramente viável. Desse modo, uma classificação binária entre *infants* e adultos surge como uma alternativa coerente, capaz de refletir as diferenças reais observadas na morfologia dos moluscos.

# Classificação Adultos e Infants

##Preparo dos Dados


Agrupamento de Femeas e Machos na Categoria Adultos, facilitando a análise.

In [79]:
y = abalone_df["Sex"]
y = pd.get_dummies(y)
y["A"] = y["F"] + y["M"]
y = y.drop(columns=["F", "M"])

In [80]:
y.head(10)

,I,A
0,False,True
1,False,True
2,False,True
3,False,True
4,True,False
5,True,False
6,False,True
7,False,True
8,False,True
9,False,True


Esse comando me permite confirmar se y está armazenado como um vetor unidimensional — formato esperado pelos classificadores — ou se ainda mantém a estrutura resultante do one-hot encoding, que gera uma matriz com uma coluna para cada classe.

Em seguida, incluí uma verificação para corrigir automaticamente esse problema caso ele ocorra

In [81]:
if len(y.shape) > 1:
    y = np.argmax(y, axis=1)
print("y shape:", y.shape)

y shape: (4177,)


##Regressão Logística

In [86]:
def treinar_logistic_regression(X, y):
    # Divisão treino-teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # Modelo base
    log_reg = LogisticRegression(max_iter=1000, random_state=42)

    # Grade de hiperparâmetros
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    }

    # Validação cruzada
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Grid Search
    grid_search = GridSearchCV(
        estimator=log_reg,
        param_grid=param_grid,
        scoring='accuracy',
        cv=cv,
        verbose=1,
        n_jobs=-1
    )

    # Ajuste do modelo
    grid_search.fit(X_train, y_train)

    # Melhor modelo
    best_model = grid_search.best_estimator_

    # Avaliação no teste
    y_pred = best_model.predict(X_test)

    print("Melhores hiperparâmetros:", grid_search.best_params_)
    print("Acurácia no conjunto de teste:", accuracy_score(y_test, y_pred))

    return best_model, accuracy_score(y_test, y_pred)

resultado= treinar_logistic_regression(X, y)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Melhores hiperparâmetros: {'C': 100, 'solver': 'sag'}
Acurácia no conjunto de teste: 0.8265550239234449


O modelo de regressão logística binária apresentou desempenho sólido na tarefa de distinguir entre abalones adultos e jovens (infants). Após a busca em grade, os melhores hiperparâmetros identificados foram C = 100 e solver = 'sag', indicando que o modelo se beneficia de uma regularização mais fraca e de um método de otimização eficiente para grandes conjuntos de dados.

A acurácia obtida no conjunto de teste foi de 0,8266, o que revela uma capacidade consistente de separação entre as duas classes. Esse desempenho confirma que as diferenças físicas entre animais jovens e adultos são suficientemente marcantes para permitir uma classificação eficaz, especialmente quando comparadas às diferenças sutis entre machos e fêmeas observadas anteriormente.


## Random Forest

In [90]:
def treinar_random_forest(X, y, test_size=0.2, random_state=42):

    # Divisão treino–teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    # Modelo base
    rf = RandomForestClassifier(random_state=random_state)

    # Grade de hiperparâmetros
    param_grid_rf = {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10],
    }

    # Validação cruzada estratificada
    cv_rf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    # Grid Search
    grid_search_rf = GridSearchCV(
        estimator=rf,
        param_grid=param_grid_rf,
        scoring="accuracy",
        cv=cv_rf,
        verbose=1,
        n_jobs=-1,
    )

    # Ajuste do modelo
    grid_search_rf.fit(X_train, y_train)

    # Melhor modelo obtido
    best_rf_model = grid_search_rf.best_estimator_

    # Predições
    y_pred_rf = best_rf_model.predict(X_test)

    # Acurácia no teste
    test_accuracy = accuracy_score(y_test, y_pred_rf)

    print("Melhores hiperparâmetros:", grid_search_rf.best_params_)
    print("Acurácia no conjunto de teste:", test_accuracy)

    return best_rf_model, test_accuracy, y_pred_rf

resultado= treinar_random_forest(X, y)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Melhores hiperparâmetros: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
Acurácia no conjunto de teste: 0.8504784688995215


O modelo de Random Forest apresentou desempenho ligeiramente superior ao da regressão logística na tarefa de distinguir abalones adultos de indivíduos jovens.

Com base na busca em grade, os melhores hiperparâmetros encontrados foram max_depth = 10, min_samples_split = 5 e n_estimators = 200, indicando que um número maior de árvores combinado a uma profundidade moderada favoreceu a capacidade de generalização do modelo.

 A acurácia obtida no conjunto de teste foi de 0.85, sugerindo que o algoritmo consegue capturar, de forma mais eficaz, padrões não lineares presentes nas características físicas que diferenciam juvenis de adultos. Esse resultado reforça a vantagem dos métodos baseados em comitês de árvores para lidar com interações e estruturas mais complexas nos dados, entregando uma classificação mais robusta nesse tipo de problema.


## Conclusão

A acurácia dos modelos binários, tanto a regressão logística quanto o random forest, mostrou-se significativamente superior à obtida nas tarefas anteriores, o que era esperado dada a maior separabilidade entre indivíduos jovens (infants) e adultos.

Mesmo que a definição prática dessa divisão seja um pouco subjetiva, os resultados indicam que características físicas simples, como altura, peso e diâmetro, carregam informação suficiente para discriminar bem essas duas classes, dispensando a análise direta dos órgãos reprodutivos.

Isso reforça que a diferença morfológica associada à imaturidade é substancial e capturável pelos modelos, permitindo classificações confiáveis a partir de atributos não invasivos.
